In [20]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the data
data = pd.read_csv('combined_data.csv', delimiter=';')

# Initialize LabelEncoder
le = LabelEncoder()

# Encode each categorical column
for column in data.columns[:-1]:  # Exclude the 'Label' column
    data[column] = le.fit_transform(data[column].astype(str))

# Encode the labels as well
data['Label'] = le.fit_transform(data['Label'].astype(str))

# Split the data into features (X) and target (y)
X = data.iloc[:, :-1].values  # All columns except the last one
y = data['Label'].values      # The last column is the target

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model using Keras
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

# Save the model in TensorFlow SavedModel format
model.export('tensorflow_model')
model.save('tensorflow_model.h5')


Epoch 1/10


/Users/berkebozaci/.pyenv/versions/3.11.3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2829 - loss: 2.6144 - val_accuracy: 0.7690 - val_loss: 1.0771
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 0.8110 - loss: 0.8355 - val_accuracy: 0.8660 - val_loss: 0.5294
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.8902 - loss: 0.4268 - val_accuracy: 0.8920 - val_loss: 0.3756
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 0.9052 - loss: 0.3330 - val_accuracy: 0.9040 - val_loss: 0.3101
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 648us/step - accuracy: 0.9179 - loss: 0.2640 - val_accuracy: 0.8930 - val_loss: 0.2968
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - accuracy: 0.9129 - loss: 0.2489 - val_accuracy: 0.9200 - val_loss: 0.2495
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - accuracy: 0.9250 - loss: 0.2224 - val_accuracy: 0.9050 - val_loss: 0.2468
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 660us/step - accuracy: 0.9362 - loss: 0.1931 - val_accuracy:

INFO:tensorflow:Assets written to: tensorflow_model/assets


Saved artifact at 'tensorflow_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='keras_tensor_48')
Output Type:
  TensorSpec(shape=(None, 24), dtype=tf.float32, name=None)
Captures:
  5522103312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5522099664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5522100240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5522102544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5522102352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5522103696: TensorSpec(shape=(), dtype=tf.resource, name=None)


enter the command to convert `tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --signature_name=serving_default --saved_model_tags=serve tensorflow_model/ model_tfjs/`


In [21]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the model from the .h5 file
model = tf.keras.models.load_model('tensorflow_model.h5')

# Load the dataset to fit the LabelEncoder (used during training)
data = pd.read_csv('combined_data.csv', delimiter=';')
le = LabelEncoder()
le.fit(data['Label'])


LabelEncoder()

In [19]:
# Create mock data - a single example with the same shape as the training data
mock_data = np.array([[0, 4, 1, 2, 0, 2, 0, 2, 0, 2]])  # Example input

# Predict the class for the mock data
prediction = model.predict(mock_data)

# Get the predicted class index
predicted_class = np.argmax(prediction, axis=-1)

# Decode the predicted class index to the ASL letter
predicted_label = le.inverse_transform(predicted_class)

print(f'Predicted class index: {predicted_class[0]}')
print(f'Predicted ASL letter: {predicted_label[0]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted class index: 15
Predicted ASL letter: Q
